### Data cleaning

In [2]:
import pandas as pd
import numpy as np
from ccxt.static_dependencies.lark.parsers.cyk import print_parse

# 1--DATA CLEANING
# region

# read 48 industry file
file_path = '48_industry_portfolios.xlsx'

# Read each sheet into a separate DataFrame
df_number_firms = pd.read_excel(file_path, sheet_name='number_firms', index_col=0)
df_firm_size = pd.read_excel(file_path, sheet_name='firm_size', index_col=0)
df_sum_BE = pd.read_excel(file_path, sheet_name='sum_BE', index_col=0)
df_avg_ind_rets = pd.read_excel(file_path, sheet_name='avg_ind_rets', index_col=0)

# Convert index to string format YYYY-MM
for df in [df_number_firms, df_firm_size,df_sum_BE, df_avg_ind_rets]:
    df.index = df.index.astype(str)
    df.index = df.index.str[:4] + '-' + df.index.str[4:]

# Convert index to string format YYYY
df_sum_BE
df_sum_BE.index = df_sum_BE.index.astype(str)
df_sum_BE.index = df_sum_BE.index.str[:4]

# endregion


### Standardize factor data 

In [3]:

# 1.1--MARKET CAP
# region
df_market_cap = df_firm_size * df_number_firms

# endregion

# 1.2--BOOK TO MARKET RATIO
# region

# Expand annual Book-to-Market ratio (Sum BE / Sum ME) to monthly values
bm_monthly = []
for year in df_sum_BE.index.astype(int):
    for month in range(7, 13):  # July to December of year s
        bm_monthly.append([f"{year}-{month:02d}"] + list(df_sum_BE.loc[str(year)]))
    for month in range(1, 7):  # January to June of year s+1
        bm_monthly.append([f"{year+1}-{month:02d}"] + list(df_sum_BE.loc[str(year)]))

# Convert to DataFrame with YYYY-MM index
df_BM_monthly = pd.DataFrame(bm_monthly, columns=["Date"] + list(df_sum_BE.columns))
df_BM_monthly.set_index("Date", inplace=True)

# Ensure df_BM_monthly index matches df_market_cap index
df_BM_monthly = df_BM_monthly.loc[df_market_cap.index]

# Compute monthly Book-to-Market ratio for each industry
df_book_to_market = df_BM_monthly * df_market_cap  # Element-wise multiplication

# endregion

# 1.3--MOMENTUM
# region
# Compute momentum for each industry
df_momentum = pd.DataFrame(index=df_avg_ind_rets.index, columns=df_avg_ind_rets.columns)

# Iterate over each month and industry to compute momentum
for date in df_avg_ind_rets.index:
    # Get the last 12 months' data up to the current month
    momentum_data = df_avg_ind_rets.loc[df_avg_ind_rets.index <= date].tail(12)

    # Calculate the average of the last 12 months for each industry
    df_momentum.loc[date] = momentum_data.mean(axis=0)

# endregion


In [4]:
# 2--STANDARDIZE DATA

def standardize_monthly(df):
    df_standardized = pd.DataFrame(index=df.index, columns=df.columns)

    for date in df.index:
        values = df.loc[date]
        # Exclude invalid values
        valid_values = values[(values != -99.99) & (values != -0.0)]

        mean_val = valid_values.mean()
        std_val = valid_values.std()

        # Standardize only valid values
        standardized_values = (valid_values - mean_val) / std_val

        # Store back
        df_standardized.loc[date, standardized_values.index] = standardized_values

    return df_standardized

# Apply standardization
standardized_market_cap = standardize_monthly(df_market_cap)
standardized_book_to_market = standardize_monthly(df_book_to_market)
standardized_momentum = standardize_monthly(df_momentum)

pd.set_option('display.max_columns', None)
print(standardized_market_cap)


            Agric     Food      Soda      Beer      Smoke     Toys      Fun    \
1926-07 -0.340402  0.536441       NaN -0.597518  0.266341 -0.605249 -0.458363   
1926-08 -0.345924  0.507911       NaN -0.605694  0.254909 -0.611264 -0.464158   
1926-09 -0.344016  0.508245       NaN -0.597739  0.284339   -0.6059 -0.466267   
1926-10 -0.350338  0.521362       NaN -0.601232  0.292081 -0.609133 -0.461784   
1926-11 -0.342589  0.508503       NaN -0.600176  0.330109 -0.607725 -0.464402   
...           ...       ...       ...       ...       ...       ...       ...   
2024-08   -0.5122 -0.327631 -0.359244 -0.357932 -0.406067 -0.515784 -0.308447   
2024-09 -0.513145 -0.322565 -0.354046 -0.360595 -0.399325 -0.517539 -0.300188   
2024-10 -0.510291 -0.323462  -0.35529 -0.360884 -0.404423 -0.514543 -0.293384   
2024-11 -0.505333 -0.328809 -0.364867 -0.365679 -0.389621 -0.510462 -0.279833   
2024-12 -0.510628 -0.343708 -0.378899 -0.380593  -0.39902 -0.516123 -0.264912   

            Books     Hshld

### Market cap portfolio (monthly)

In [5]:
# 3-- MARKET CAP PF (w_i)
df_market_cap_portfolio = df_market_cap.div(df_market_cap.sum(axis=1), axis=0)

### In sample estimate of theta

In [6]:
# 4-- IN-SAMPLE ESTIMATE OF THETA
# region

from scipy.optimize import minimize

# Define CRRA utility with risk aversion 5
def crra_utility(r, gamma=5):
    if gamma == 1:
        return np.log(1 + r)
    else:
        return ( (1 + r) ** (1 - gamma) ) / (1 - gamma)

# Objective function (minimize negative average utility)
def objective(theta, dates=None):
    theta_size, theta_bm, theta_mom = theta

    if dates is None:
        dates = df_avg_ind_rets.index[:-1]

    r_p_array = []

    for t in range(len(dates)):
        date_t = dates[t]
        date_t1 = df_avg_ind_rets.index[df_avg_ind_rets.index.get_loc(date_t) + 1]

        w_hat = df_market_cap_portfolio.loc[date_t].astype(float)
        x_size = standardized_market_cap.loc[date_t].astype(float)
        x_bm = standardized_book_to_market.loc[date_t].astype(float)
        x_mom = standardized_momentum.loc[date_t].astype(float)

        tilt = (theta_size * x_size + theta_bm * x_bm + theta_mom * x_mom) / len(w_hat.dropna())
        weights = w_hat + tilt

        r_next = df_avg_ind_rets.loc[date_t1].astype(float)
        r_p = np.dot(weights.fillna(0).values, r_next.fillna(0).values)

        r_p_array.append(r_p)

    utilities = crra_utility(np.array(r_p_array), gamma=5)
    return -np.mean(utilities)

# Optimization with stricter stopping and lower tolerance for speed
initial_guess = [0.0, 0.0, 0.0]
result = minimize(
    objective,
    initial_guess,
    method='Nelder-Mead',
    options={'maxiter': 300, 'fatol': 1e-4, 'disp': True}
)

theta_estimate = result.x
# Estimates of theta
initial_guess = [0.0, 0.0, 0.0]
result = minimize(objective, initial_guess, method='Nelder-Mead')
theta_estimate = result.x
print(theta_estimate)

Optimization terminated successfully.
         Current function value: 17951.323191
         Iterations: 101
         Function evaluations: 186
[-0.03474951  0.015726    0.01253649]


### Out of sample estimate of theta + description statistics of backtest

In [7]:
# 5-- OUT OF SAMPLE BACKTEST

def run_out_of_sample_backtest(start_year='1974-01'):
    results = []
    dates = df_avg_ind_rets.index

    for year in range(1974, int(dates[-1][:4])):
        print(f"Processing year: {year}") # print year processing to track code progression
        estimation_dates = dates[dates < f"{year}-01"]
        oos_dates = dates[(dates >= f"{year}-01") & (dates < f"{year+1}-01")]

        # Estimate theta using data up to end of prior year
        res = minimize(objective, [0.0, 0.0, 0.0], args=(estimation_dates,), method='Nelder-Mead')
        theta_size, theta_bm, theta_mom = res.x

        # Out-of-sample portfolio formation
        for date_t in oos_dates[:-1]:
            date_t1 = dates[dates > date_t][0]

            w_hat = df_market_cap_portfolio.loc[date_t].astype(float)
            x_size = standardized_market_cap.loc[date_t].astype(float)
            x_bm = standardized_book_to_market.loc[date_t].astype(float)
            x_mom = standardized_momentum.loc[date_t].astype(float)

            tilt = (theta_size * x_size + theta_bm * x_bm + theta_mom * x_mom) / len(w_hat.dropna())
            weights = w_hat + tilt

            # Renormalize weights according to equation (16)
            weights = weights / weights.sum()

            r_next = df_avg_ind_rets.loc[date_t1].astype(float)
            r_p = (weights * r_next).sum()

            results.append({'date': date_t1, 'portfolio_return': r_p})

    result_df = pd.DataFrame(results)
    result_df.set_index('date', inplace=True)

    # Compute performance metrics
    mean_monthly = result_df['portfolio_return'].mean()
    std_monthly = result_df['portfolio_return'].std()
    annualized_return = mean_monthly * 12
    annualized_std = std_monthly * np.sqrt(12)
    sharpe_ratio = annualized_return / annualized_std

    return result_df, annualized_return, annualized_std, sharpe_ratio

# Run out-of-sample backtest
out_sample_results, annual_ret, annual_std, sharpe = run_out_of_sample_backtest()
print('results',out_sample_results)
print('annual rets', annual_ret)
print('annual std',annual_std)
print('sharpe', sharpe)

Processing year: 1974
Processing year: 1975
Processing year: 1976
Processing year: 1977
Processing year: 1978
Processing year: 1979
Processing year: 1980
Processing year: 1981
Processing year: 1982
Processing year: 1983
Processing year: 1984
Processing year: 1985
Processing year: 1986
Processing year: 1987
Processing year: 1988
Processing year: 1989
Processing year: 1990
Processing year: 1991
Processing year: 1992
Processing year: 1993
Processing year: 1994
Processing year: 1995
Processing year: 1996
Processing year: 1997
Processing year: 1998
Processing year: 1999
Processing year: 2000
Processing year: 2001
Processing year: 2002
Processing year: 2003
Processing year: 2004
Processing year: 2005
Processing year: 2006
Processing year: 2007
Processing year: 2008
Processing year: 2009
Processing year: 2010
Processing year: 2011
Processing year: 2012
Processing year: 2013
Processing year: 2014
Processing year: 2015
Processing year: 2016
Processing year: 2017
Processing year: 2018
Processing